# Soving direct and inverse problems of a single stage refrigeration vapor-comression cycle
### Demonstration of the Thermal-Hydraulic Computation Framework

This notebook demonstrates the solution of a steady-state heat pump
network using the framework introduced in:

> M. Ganz (2025)  
> *Computational Modelling Framework for Solving Direct and Inverse Problems  
> in Steady-State Thermal-Hydraulic Network Calculations*

The example covers:
- automatic network topology construction
- tripartite graph formulation
- tearing-based execution order
- loop breaking in closed fluid cycles
- direct simulation
- inverse optimization


In [20]:
# import the framework
from thdin import *

## 1. Problem Description

We consider a steady-state heat pump consisting of:

- Evaporator
- Compressor
- Condenser
- Expansion valve
- Internal heat exchanger (IHX)
- Receiver

The system includes:
- one closed refrigerant loop (R134a)
- two secondary incompressible loops (antifrogen)

Both **direct** (forward simulation) and **inverse** (parameter estimation /
design optimization) problems are solved using the same network formulation.


In [21]:
net = Network()

net.fluid_loops[1] = "R134a"
net.fluid_loops[2] = "INCOMP::AN[0.4]"
net.fluid_loops[3] = "INCOMP::AN[0.4]"


## 2. Component Models

Each component is defined by:
- a set of ports
- a causal modeling type (pressure-based / mass-flow-based)
- a local component model

The global causality is resolved automatically via tearing.


In [22]:
evap = MassFlowBasedComponent(
    label="Evap",
    port_specs={
        "in_hot":  PortSpec("in"),
        "out_hot": PortSpec("out"),
        "in_cold": PortSpec("in"),
        "out_cold":PortSpec("out"),
    },
)

comp = PressureBasedComponent(
    label="Comp",
    port_specs={
        "in":  PortSpec("in"),
        "out": PortSpec("out"),
    },
)

cond = MassFlowBasedComponent(
    label="Cond",
    port_specs={
        "in_hot":  PortSpec("in"),
        "out_hot": PortSpec("out"),
        "in_cold": PortSpec("in"),
        "out_cold":PortSpec("out"),
    },
)

exp = PressureBasedComponent(
    label="Exp",
    port_specs={
        "in":  PortSpec("in"),
        "out": PortSpec("out"),
    },
)

rev = MassFlowBasedComponent(
    label="Rev",
    port_specs={
        "in":  PortSpec("in"),
        "out": PortSpec("out"),
    },
)

ihx = MassFlowBasedComponent(
    label="IHX",
    port_specs={
        "in_hot":  PortSpec("in"),
        "out_hot": PortSpec("out"),
        "in_cold": PortSpec("in"),
        "out_cold":PortSpec("out"),
    },
)

net.add_component(comp)
net.add_component(cond)
net.add_component(evap)
net.add_component(exp)
net.add_component(ihx)
net.add_component(rev)


In [23]:
def evap_model(evap):

    N = 50
    k = 1000

    A = evap.parameter['A'].value

    hi_h = evap.ports['in_hot'].h.value
    pi_h = evap.ports['in_hot'].p.value
    m_h = evap.ports['in_hot'].m.value
    hot_fluid = evap.ports['in_hot'].fluid

    hi_c = evap.ports['in_cold'].h.value
    pi_c = evap.ports['in_cold'].p.value
    m_c = evap.ports['in_cold'].m.value
    cold_fluid = evap.ports['in_cold'].fluid

    def res(x):
        h_h = np.zeros(N)
        h_c = np.zeros(N)
        h_c[0] = hi_c
        h_h[0] = x[0] * 1e5
        for i in range(N - 1):
            Th = PropsSI('T', 'H', h_h[i], 'P', pi_h, hot_fluid)
            Tc = PropsSI('T', 'H', h_c[i], 'P', pi_c, cold_fluid)
            h_h[i + 1] = h_h[i] + k * A / N * (Th - Tc) / m_h
            h_c[i + 1] = h_c[i] + k * A / N * (Th - Tc) / m_c
        return np.array([(h_h[-1] - hi_h) * 1e-5])

    Ti_c = PropsSI('T', 'H', hi_c, 'P', pi_c, cold_fluid)
    ho_h_init = PropsSI('H', 'P', pi_h, 'T', max(Ti_c, 257.0), hot_fluid) * 1e-5
    sol = scipy.optimize.root(res, np.array([ho_h_init]), method='hybr', tol=1e-5)

    if sol.success:
        ho_h = sol.x[0] * 1e5
        h_h = np.zeros(N)
        h_c = np.zeros(N)
        h_c[0] = hi_c
        h_h[0] = ho_h
        for i in range(N - 1):
            Th = PropsSI('T', 'H', h_h[i], 'P', pi_h, hot_fluid)
            Tc = PropsSI('T', 'H', h_c[i], 'P', pi_c, cold_fluid)
            h_h[i + 1] = h_h[i] + k * A / N * (Th - Tc) / m_h
            h_c[i + 1] = h_c[i] + k * A / N * (Th - Tc) / m_c
    else:
        raise Exception(sol.message)

    po_h = pi_h
    ho_h = h_h[0]
    mo_h = m_h

    po_c = pi_c
    ho_c = h_c[-1]
    mo_c = m_c

    evap.ports['out_hot'].p.set_value(po_h)
    evap.ports['out_hot'].h.set_value(ho_h)
    evap.ports['out_hot'].m.set_value(mo_h)

    evap.ports['out_cold'].p.set_value(po_c)
    evap.ports['out_cold'].h.set_value(ho_c)
    evap.ports['out_cold'].m.set_value(mo_c)

    evap.outputs["Q"].set_value(m_c * (ho_c - hi_c))


def comp_model(comp):

    n = 4 # number of cylinders
    bore = 0.06 # bore [m]
    stroke = 0.042 # stroke [m]
    Vd = n * bore**2 * np.pi / 4 * stroke # displacement volume

    a = np.array([-5.31166292e-02, 1.21402922e-03, 8.81226071e-05, 1.03163725e+00])
    b = np.array([9.38116126e-03, -1.52858792e-03, -4.08026601e-03, 6.31332600e-04, 6.77625196e-01])

    RPM = comp.parameter['RPM'].value
    hi = comp.ports['in'].h.value
    pi = comp.ports['in'].p.value
    po = comp.ports['out'].p.value
    fluid = comp.ports['in'].fluid

    eta_v = a[0] * (po / pi) + a[1] * (po / pi) ** 2 + a[2] * (RPM / 60) + a[3]
    eta_is = b[0] * (po / pi) + b[1] * (po / pi) ** 2 + b[2] * (RPM / 60) + b[3] * (po / pi) * (RPM / 60) + b[4]

    s = PropsSI('S', 'P', pi, 'H', hi, fluid)
    h_is = PropsSI('H', 'P', po, 'S', s, fluid)
    rho = PropsSI('D', 'P', pi, 'H', hi, fluid)
    ho = hi + (h_is - hi) / eta_is
    mi = mo = RPM / 60 * Vd * eta_v * rho

    comp.ports['in'].m.set_value(mi)
    comp.ports['out'].h.set_value(ho)
    comp.ports['out'].m.set_value(mo)


def cond_model(cond):

    N = 50
    k = 1000

    A = cond.parameter['A'].value

    hi_h = cond.ports["in_hot"].h.value
    pi_h = cond.ports["in_hot"].p.value
    m_h = cond.ports["in_hot"].m.value
    hot_fluid = cond.ports["in_hot"].fluid

    hi_c = cond.ports["in_cold"].h.value
    pi_c = cond.ports["in_cold"].p.value
    m_c = cond.ports["in_cold"].m.value
    cold_fluid = cond.ports["in_cold"].fluid

    def res(x):
        h_h = np.zeros(N)
        h_c = np.zeros(N)
        h_h[0] = hi_h
        h_c[0] = x[0] * 1e5
        for i in range(N - 1):
            Th = PropsSI('T', 'H', h_h[i], 'P', pi_h, hot_fluid)
            Tc = PropsSI('T', 'H', h_c[i], 'P', pi_c, cold_fluid)
            h_h[i + 1] = h_h[i] - k * A / N * (Th - Tc) / m_h
            h_c[i + 1] = h_c[i] - k * A / N * (Th - Tc) / m_c
        return np.array([(h_c[-1] - hi_c) * 1e-5])

    Ti_h = PropsSI('T', 'H', hi_h, 'P', pi_h, hot_fluid)
    ho_c_init = PropsSI('H', 'P', pi_c, 'T', min(Ti_h, 350.0), cold_fluid) * 1e-5
    sol = scipy.optimize.root(res, np.array([ho_c_init]), method='hybr', tol=1e-5)

    if sol.success:
        ho_c = sol.x[0] * 1e5
        h_h = np.zeros(N)
        h_c = np.zeros(N)
        h_h[0] = hi_h
        h_c[0] = ho_c
        for i in range(N - 1):
            Th = PropsSI('T', 'H', h_h[i], 'P', pi_h, hot_fluid)
            Tc = PropsSI('T', 'H', h_c[i], 'P', pi_c, cold_fluid)
            h_h[i + 1] = h_h[i] - k * A / N * (Th - Tc) / m_h
            h_c[i + 1] = h_c[i] - k * A / N * (Th - Tc) / m_c
    else:
        raise Exception(sol.message)

    po_h = pi_h
    ho_h = h_h[-1]
    mo_h = m_h

    po_c = pi_c
    ho_c = h_c[0]
    mo_c = m_c

    cond.ports["out_hot"].p.set_value(po_h)
    cond.ports["out_hot"].h.set_value(ho_h)
    cond.ports["out_hot"].m.set_value(mo_h)

    cond.ports["out_cold"].p.set_value(po_c)
    cond.ports["out_cold"].h.set_value(ho_c)
    cond.ports["out_cold"].m.set_value(mo_c)


def exp_model(exp):

    Cf = exp.parameter['Cf'].value
    Av = exp.parameter['Av'].value
    hi = exp.ports["in"].h.value
    pi = exp.ports["in"].p.value
    po = exp.ports["out"].p.value
    fluid = exp.ports["in"].fluid

    rho = PropsSI('D', 'P', pi, 'H', hi, fluid)
    pressure_difference = (pi - po) * 1e-5
    n = pressure_difference * 1000 * rho
    m = Cf * Av * np.sqrt(n) / 3600
    ho = hi
    mi = mo = m

    exp.ports["in"].m.set_value(mi)
    exp.ports["out"].h.set_value(ho)
    exp.ports["out"].m.set_value(mo)


def rev_model(rev):

    p_in = rev.ports["in"].p.value
    m_in = rev.ports["in"].m.value
    fluid = rev.ports["in"].fluid
    p_out = p_in
    h_out = PropsSI('H', 'P', p_in, 'Q', 0.0, fluid)
    m_out = m_in
    rev.ports["out"].p.set_value(p_out)
    rev.ports["out"].h.set_value(h_out)
    rev.ports["out"].m.set_value(m_out)


def ihx_model(ihx):

    k = 1000

    A = ihx.parameter['A'].value

    hi_h = ihx.ports["in_hot"].h.value - 1e-3
    pi_h = ihx.ports["in_hot"].p.value
    mi_h = ihx.ports["in_hot"].m.value
    hot_fluid = ihx.ports["in_hot"].fluid

    hi_c = ihx.ports["in_cold"].h.value + 1e-3
    pi_c = ihx.ports["in_cold"].p.value
    mi_c = ihx.ports["in_cold"].m.value
    cold_fluid = ihx.ports["in_cold"].fluid

    C_h = mi_h * PropsSI('C', 'P', pi_h, 'H', hi_h, hot_fluid)
    C_c = mi_c * PropsSI('C', 'P', pi_c, 'H', hi_c, cold_fluid)
    T_in_h = PropsSI('T', 'P', pi_h, 'H', hi_h, hot_fluid)
    T_in_c = PropsSI('T', 'P', pi_c, 'H', hi_c, cold_fluid)

    R = C_h / C_c
    NTU = k * A / C_h
    n = (-1) * NTU * (1 + R)
    epsilon = (1 - np.exp(n)) / (1 + R)
    Q = epsilon * C_h * (T_in_h - T_in_c)

    po_c = pi_c
    po_h = pi_h
    ho_h = (hi_h - Q / mi_h)
    ho_c = (hi_c + Q / mi_c)
    mo_h = mi_h
    mo_c = mi_c

    ihx.ports["out_hot"].p.set_value(po_h)
    ihx.ports["out_hot"].h.set_value(ho_h)
    ihx.ports["out_hot"].m.set_value(mo_h)

    ihx.ports["out_cold"].p.set_value(po_c)
    ihx.ports["out_cold"].h.set_value(ho_c)
    ihx.ports["out_cold"].m.set_value(mo_c)


net.set_component_model("Comp", comp_model)
net.set_component_model("Cond", cond_model)
net.set_component_model("Evap", evap_model)
net.set_component_model("Exp", exp_model)
net.set_component_model("IHX", ihx_model)
net.set_component_model("Rev", rev_model)

## 3. Network Topology

Connections define the topology only.

In [24]:
net.connect("IHX", "out_cold", "Comp", "in",  fluid_loop=1)
net.connect("Comp", "out", "Cond", "in_hot",   fluid_loop=1)
net.connect("Cond", "out_hot", "Rev", "in", fluid_loop=1)
net.connect("Rev", "out", "IHX", "in_hot", fluid_loop=1)
net.connect("IHX", "out_hot", "Exp", "in", fluid_loop=1)
net.connect("Exp", "out", "Evap", "in_cold", fluid_loop=1)
net.connect("Evap", "out_cold", "IHX", "in_cold", fluid_loop=1)


## 4. Boundary Conditions

Boundary conditions are applied at ports that are not connected
via junctions. These variables are treated as known quantities.


In [25]:
hi_evap_af = PropsSI("H", "P", 1e5, "T", -5.0+273.15, "INCOMP::AN[0.4]")
net.set_bc("Evap", "in_hot", "p", 1e5, fluid_loop=2)
net.set_bc("Evap", "in_hot", "h", hi_evap_af, fluid_loop=2)
net.set_bc("Evap", "in_hot", "m", 1.0, fluid_loop=2)

hi_cond_af = PropsSI("H", "P", 1e5, "T", 27.0+273.15, "INCOMP::AN[0.4]")
net.set_bc("Cond", "in_cold", "p", 1e5, fluid_loop=3)
net.set_bc("Cond", "in_cold", "h", hi_cond_af, fluid_loop=3)
net.set_bc("Cond", "in_cold", "m", 1.0, fluid_loop=3)

## 5. Direct and Inverse Problem Formulation

Design variables and objectives are introduced without changing
the network topology or equations.


In [26]:
net.add_parameter("Comp", "RPM", value=1500, initial_value=2400, is_var=True, scale_factor=1e-3, bounds=(750, 3500))
net.add_parameter("Cond", "A", value=3.65)
net.add_parameter("Evap", "A", value=2.56)
net.add_parameter("IHX", "A", value=0.75)
net.add_parameter("Exp", "Cf", value=0.25)
net.add_parameter("Exp", "Av", value=0.3, initial_value=0.3, is_var=True, bounds=(0.1, 1))

net.add_output(comp_label='Evap', output_label='Q')


In [27]:
def subcooling_fun(net):
    h_s = PropsSI('H', 'P', net.components['Cond'].ports["out_hot"].p.value,
                  'Q', 0.0, net.components['Cond'].ports["out_hot"].fluid)
    res = (net.components['Cond'].ports["out_hot"].h.value - h_s)
    return res

def superheat_fun(net):
    SH_value = 5.0
    T_sat = PropsSI('T', 'P', net.components['Evap'].ports["out_cold"].p.value,
                    'Q', 1.0, net.components['Evap'].ports["out_cold"].fluid)
    h_SH = PropsSI('H', 'T', T_sat + SH_value,
                   'P', net.components['Evap'].ports["out_cold"].p.value,
                   net.components['Evap'].ports["out_cold"].fluid)
    res = np.abs(net.components['Evap'].ports["out_cold"].h.value - h_SH)
    return res

def cooling_load(net):
    Qsp = 18000
    Q = net.components['Evap'].outputs['Q'].value
    res = np.abs(Q - Qsp)
    return res

net.add_constraint('subcool_eq', subcooling_fun, ctype='eq', scale_factor=1e-5)
net.add_constraint('supheat_eq', superheat_fun, ctype='obj', weight_factor=1e-5)
net.add_constraint('cool_load_eq', cooling_load, ctype='obj', weight_factor=1e-3)

## 6. Loop Breaking and Initialization

One mass balance equation is removed from the closed refrigerant loop
to avoid linear dependency.


In [28]:
net.set_loop_breaker(fluid_loop=1, junction_id=1)

## 7. Initialization and assigning initial values and bounds to tearing variables

.....

In [29]:
net.initialize()

net.print_tearing_variables()

Tearing variables:
p(in) at component <Comp> , port <in>
h(in) at component <Comp> , port <in>
p(out) at component <Comp> , port <out>
p(in) at component <Exp> , port <in>
h(in) at component <Exp> , port <in>
p(out) at component <Exp> , port <out>


In [30]:
# x_init = [2e5, 4e5, 1e6, 1e6, 2.5e5, 2e5]
x_init = [1.6e5, 4.1e5, 7.9e5, 7.9e5, 2.2e5, 1.6e5]
x_bnds = [(1e5, 4e5), (2e5, 6e5), (5e5, 30e5), (5e5, 30e5), (1e5, 3.5e5), (1e5, 4e5)]

net.set_inital_values(x_init, x_bnds)

In [31]:
net.solve_system_notebook()


 Start solver...
Residual norm = 1.720e+00, Obj = 2.321e+01
Residual norm = 3.898e-01, Obj = 6.260e+00
Residual norm = 7.931e-01, Obj = 2.407e+00
Residual norm = 7.310e-03, Obj = 1.562e-01
Residual norm = 2.046e-01, Obj = 1.942e+00
Residual norm = 1.776e+00, Obj = 1.516e+01
Residual norm = 6.509e-01, Obj = 6.181e+00
Residual norm = 7.246e-01, Obj = 1.361e+01
Residual norm = 1.432e+00, Obj = 1.719e+01
Residual norm = 2.947e-04, Obj = 6.241e-03
Residual norm = 3.128e-07, Obj = 2.987e-03
Residual norm = 1.919e-07, Obj = 2.596e-03
Residual norm = 5.938e-08, Obj = 6.304e-04
Residual norm = 4.653e-07, Obj = 1.398e-03
Residual norm = 1.109e-08, Obj = 1.503e-04
Residual norm = 2.717e-09, Obj = 1.845e-04
Residual norm = 1.428e-08, Obj = 1.612e-03
Residual norm = 1.411e-10, Obj = 3.933e-05
Residual norm = 5.985e-10, Obj = 5.438e-05
Residual norm = 2.723e-11, Obj = 1.929e-05
Residual norm = 2.188e-12, Obj = 2.079e-05
Residual norm = 5.655e-12, Obj = 8.444e-06
Residual norm = 5.900e-11, Obj = 1.5

In [32]:
res = net.get_results()

pi_comp = res["ports"]["Comp"]["in"]["p"]
po_comp = res["ports"]["Comp"]["out"]["p"]
hi_comp = res["ports"]["Comp"]["in"]["h"]
ho_comp = res["ports"]["Comp"]["out"]["h"]
mi_comp = res["ports"]["Comp"]["in"]["m"]
mo_comp = res["ports"]["Comp"]["out"]["m"]

pi_r_cond = res["ports"]["Cond"]["in_hot"]["p"]
po_r_cond = res["ports"]["Cond"]["out_hot"]["p"]
hi_r_cond = res["ports"]["Cond"]["in_hot"]["h"]
ho_r_cond = res["ports"]["Cond"]["out_hot"]["h"]
mi_r_cond = res["ports"]["Cond"]["in_hot"]["m"]
mo_r_cond = res["ports"]["Cond"]["out_hot"]["m"]

pi_r_evap = res["ports"]["Evap"]["in_cold"]["p"]
po_r_evap = res["ports"]["Evap"]["out_cold"]["p"]
hi_r_evap = res["ports"]["Evap"]["in_cold"]["h"]
ho_r_evap = res["ports"]["Evap"]["out_cold"]["h"]
mi_r_evap = res["ports"]["Evap"]["in_cold"]["m"]
mo_r_evap = res["ports"]["Evap"]["out_cold"]["m"]

pi_exp = res["ports"]["Exp"]["in"]["p"]
po_exp = res["ports"]["Exp"]["out"]["p"]
hi_exp = res["ports"]["Exp"]["in"]["h"]
ho_exp = res["ports"]["Exp"]["out"]["h"]
mi_exp = res["ports"]["Exp"]["in"]["m"]
mo_exp = res["ports"]["Exp"]["out"]["m"]

pi_rev = res["ports"]["Rev"]["in"]["p"]
po_rev = res["ports"]["Rev"]["out"]["p"]
hi_rev = res["ports"]["Rev"]["in"]["h"]
ho_rev = res["ports"]["Rev"]["out"]["h"]
mi_rev = res["ports"]["Rev"]["in"]["m"]
mo_rev = res["ports"]["Rev"]["out"]["m"]

pi_ihx_h = res["ports"]["IHX"]["in_hot"]["p"]
po_ihx_h = res["ports"]["IHX"]["out_hot"]["p"]
hi_ihx_h = res["ports"]["IHX"]["in_hot"]["h"]
ho_ihx_c = res["ports"]["IHX"]["out_cold"]["h"]
mi_ihx_c = res["ports"]["IHX"]["in_cold"]["m"]
mo_ihx_c = res["ports"]["IHX"]["out_cold"]["m"]

print(res["ports"]["Comp"]["in"]["h"])
print(res["parameters"]["Comp"]["RPM"])
print(res["outputs"]["Evap"]["Q"])


417878.9094409433
2431.3719660304832
17999.99830411368
